# The Stack and the Heap

Both the stack and the heap are parts of memory available to your code to use at runtime, but they are structured in different ways. Pushing data to the stack is faster than allocating on the heap, Accessing data in the heap is slower than accessing data on the stack because you have to follow a pointer to get there.

## The Stack

The stack stores values in the order it gets them and removes the values in the opposite order. This is referred to as last in, first out. Think of a stack of plates: when you add more plates, you put them on top of the pile, and when you need a plate, you take one off the top. Adding or removing plates from the middle or bottom wouldn’t work as well! Adding data is called pushing onto the stack, and removing data is called popping off the stack. All data stored on the stack must have a known, fixed size.

## The Heap

Data with an unknown size at compile time or a size that might change must be stored on the heap instead. When you put data on the heap, you request a certain amount of space. The memory allocator finds an empty spot in the heap that is big enough, marks it as being in use, and returns a pointer, which is the address of that location. This process is called allocating on the heap and is sometimes abbreviated as just allocating (pushing values onto the stack is not considered allocating). Because the pointer to the heap is a known, fixed size, you can store the pointer on the stack, but when you want the actual data, you must follow the pointer. Think of being seated at a restaurant. When you enter, you state the number of people in your group, and the host finds an empty table that fits everyone and leads you there. If someone in your group comes late, they can ask where you’ve been seated to find you.



In [2]:
let mut greeting = "Hello there.";  
println!("{}", greeting);

Hello there.


In [3]:
greeting.push_str(", world."); // same error occurs with let greeting = "Hello there.";
println!("{}", greeting);

Error: no method named `push_str` found for reference `&str` in the current scope

Above, "Hello there." is a string literal and its type is &'static str. A string literal is a string slice that is statically allocated, meaning that it’s saved inside our compiled program, and exists for the entire duration it runs. The greeting binding is a reference to this statically allocated string. Any function expecting a string slice will also accept a string literal.

In [3]:
let mut s = "Hello".to_string(); // mut s: String
println!("{}", s);

s.push_str(", world.");
println!("{}", s);

Hello
Hello, world.


Above, `String` is a heap-allocated string. This string is growable, and is also guaranteed to be UTF-8. Strings are commonly created by converting from a string slice using the `to_string` method.

So, what’s the difference here? Why can String be mutated but literals cannot? The difference is in how these two types deal with memory.

    A. String Literal - A string literal is hardcoded directly into the final executable so string literals are fast and efficient.
    
    B. String Type -  to support a mutable, growable piece of text, we allocate an amount of memory on the heap, unknown at compile time, to hold the contents. This means:
                     
                    1. The memory must be requested from the memory allocator at runtime.
                    2. We need a way of returning this memory to the allocator when we’re done with our String.

Types such as integers `let x = 5;` that have a known size at compile time are stored entirely on the stack, so copies of the actual values are quick to make. That means there’s no reason we would want to prevent x from being valid after we create the variable y. In other words, there’s no difference between deep and shallow copying here, so calling clone wouldn’t do anything different from the usual shallow copying, and we can leave it out.

In [6]:
let x = 5;
let y = x;
println!("x {}", x);
println!("y {}", y);
let x = 6;
println!("x {}", x);
println!("y {}", y);

x 5
y 5
x 6
y 5


In [7]:
// with String, a shallow copy and move are made
let s1 = String::from("hello");
let s2 = s1;

println!("{}, world!", s1);

Error: borrow of moved value: `s1`

In [8]:
// the clone method
let s1 = String::from("hello");
let s2 = s1.clone(); // the heap data gets copied

println!("s1 = {}, s2 = {}", s1, s2);

s1 = hello, s2 = hello


In [12]:
fn takes_ownership(some_string: String) { // some_string comes into scope
    println!("{}", some_string);
} // Here, some_string goes out of scope and `drop` is called. The backing
  // memory is freed.

fn makes_copy(some_integer: i32) { // some_integer comes into scope
    println!("{}", some_integer);
} 

fn main() {
    let s = String::from("hello");  // s comes into scope

    takes_ownership(s);             // s's value moves into the function...
                                    // ... and so is no longer valid here

    let x = 5;                      // x comes into scope

    makes_copy(x);                  // x would move into the function,
                                    // but i32 is Copy, so it's okay to still
                                    // use x afterward

} // Here, x goes out of scope, then s. But because s's value was moved, nothing
  // special happens.

main();

hello
5


In [18]:
fn main() {
    let s = String::from("hello");  // s comes into scope
    println!("first {}", s);        // this on its own without the second, works

    takes_ownership(s);             // s's value moves into the function...
    println!("second {}", s);              // ... and so is no longer valid here

    let x = 5;                      // x comes into scope

    makes_copy(x);                  // x would move into the function,
                                    // but i32 is Copy, so it's okay to still
                                    // use x afterward

} // Here, x goes out of scope, then s. But because s's value was moved, nothing
  // special happens.

main();

Error: borrow of moved value: `s`

In [22]:
fn main() {
    let s1 = gives_ownership();         // gives_ownership moves its return
                                        // value into s1
    println!("s1 {}", s1);
    let s2 = String::from("hello");     // s2 comes into scope

    let s3 = takes_and_gives_back(s2);  // s2 is moved into
                                        // takes_and_gives_back, which also
                                        // moves its return value into s3
    println!("s3 {}", s3);
} // Here, s3 goes out of scope and is dropped. s2 was moved, so nothing
  // happens. s1 goes out of scope and is dropped.

fn gives_ownership() -> String {             // gives_ownership will move its
                                             // return value into the function
                                             // that calls it

    let some_string = String::from("yours"); // some_string comes into scope

    some_string                              // some_string is returned and
                                             // moves out to the calling
                                             // function
}

// This function takes a String and returns one
fn takes_and_gives_back(a_string: String) -> String { // a_string comes into
                                                      // scope

    a_string  // a_string is returned and moves out to the calling function
}

main();

s1 yours
s3 hello


In [23]:
fn main() {
    let s1 = String::from("hello");

    let (s2, len) = calculate_length(s1);

    println!("The length of '{}' is {}.", s2, len);
}

fn calculate_length(s: String) -> (String, usize) {
    let length = s.len(); // len() returns the length of a String

    (s, length)
}

main();

The length of 'hello' is 5.
